# Notebook 2: AI-Generated SQL Checks

## From Manual to Automated 🤖

**👋 Recap:** In Notebook 1, you wrote SQL by hand to explore data. That works for investigation, but doesn't scale.

**😫 The problem:** You need dozens of checks running daily:
- Missing consent flags?
- PII in URLs?
- Duplicate events?
- Platform-specific tracking breaks?
- Traffic spikes and new referrers?

Writing and maintaining all that SQL is tedious and error-prone.

**✨ The solution:** Use AI to generate SQL from plain English descriptions.

---

### What We'll Do

1. ✨ Generate SQL with GPT (no SQL expertise needed!)
2. 🔍 Detect the **4 planted problems** in our data
3. 🎉 Find the **2 planted opportunities**
4. 🧠 Learn prompt engineering for better SQL

**By the end:** You'll have found all 6 hidden issues in the data just by describing what to check in plain English.

Let's go! 🚀

---

## Step 1: Install & Setup

**What this does:** Installs OpenAI library + imports everything we need.

In [ ]:
# Install OpenAI package
!pip install -q openai google-cloud-bigquery pandas

print("✓ Packages installed!")

In [ ]:
# Imports
from google.cloud import bigquery
from google.colab import auth
import openai
import pandas as pd
import json

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

print("✓ Libraries imported!")

---

## Step 2: Authenticate

**What this does:** Logs you into Google Cloud (same as Notebook 1).

In [ ]:
# Authenticate with Google Cloud
auth.authenticate_user()

print("✓ Google Cloud authentication complete!")

---

## Step 3: Configure BigQuery + OpenAI

**📝 Update OpenAI API Key:**
- Get from platform.openai.com (or just watch the demo!)
- BigQuery project already configured

In [ ]:
# BigQuery configuration
PROJECT_ID = "npa-workshop-2025"  # ⬅️ Workshop project ID
DATASET_ID = "npa_workshop"
TABLE_ID = "news_events"
TABLE_REF = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"

# OpenAI configuration
OPENAI_API_KEY = "sk-your-key-here"  # ⬅️ UPDATE THIS (or just watch!)

# Initialize OpenAI client (new API v1.0+)
from openai import OpenAI
client_openai = OpenAI(api_key=OPENAI_API_KEY)

print("✓ Configuration set!")
print(f"  BigQuery table: {TABLE_REF}")
print(f"  OpenAI key: {OPENAI_API_KEY[:10]}..." if OPENAI_API_KEY.startswith('sk-') else "⚠️ No API key - watching mode only")

---

## Step 4: Connect to BigQuery

**What this does:** Establishes database connection (same as before).

In [ ]:
# Initialize BigQuery client
client = bigquery.Client(project=PROJECT_ID)

# Helper function
def run_query(sql):
    """Execute SQL and return results as DataFrame."""
    query_job = client.query(sql)
    return query_job.to_dataframe()

# Get schema for AI
table = client.get_table(TABLE_REF)
schema_info = "\n".join([f"- {field.name}: {field.field_type}" for field in table.schema])

print("✓ Connected to BigQuery!")
print(f"  Table: {table.table_id}")
print(f"  Rows: {table.num_rows:,}")

---

# The Magic: SQL Generator 🪄

**This is the key component:** A simple function that takes plain English and returns BigQuery SQL.

**How it works:**
1. You describe what to check ("Find iOS events where scroll_depth dropped")
2. We send that + the schema to GPT
3. GPT returns perfect BigQuery SQL
4. We execute it

**That's it!** No SQL expertise required.

In [ ]:
def generate_sql(description, model="gpt-4"):
    """
    Generate BigQuery SQL from plain English description.
    
    Args:
        description: Plain English check description
        model: "gpt-4" (better) or "gpt-3.5-turbo" (cheaper)
    
    Returns:
        SQL string
    """
    prompt = f"""You are a BigQuery SQL expert. Generate SQL for this analytics check.

Table: `{TABLE_REF}`

Schema:
{schema_info}

Check Description:
{description}

Requirements:
- Return ONLY valid BigQuery SQL, no explanation
- Use standard SQL syntax (not legacy)
- Include comments explaining logic
- Order results meaningfully
"""

    response = client_openai.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0  # Deterministic output
    )
    
    sql = response.choices[0].message.content
    
    # Clean up markdown code blocks if present
    if sql.startswith('```'):
        sql = sql.split('```')[1]
        if sql.startswith('sql\n'):
            sql = sql[4:]
    
    return sql.strip()

print("✓ SQL Generator function defined!")
print("\nReady to generate SQL from plain English! 🎉")

---

# Let's Find All 6 Issues! 🔍

Now we'll use the SQL generator to find:
- **4 problems** (missing consent, iOS break, PII leak, duplicates)
- **2 opportunities** (newsletter spike, new referrer)

Just by describing what to check in plain English!

---

## Check 1: Missing Consent (GDPR Risk) ⚠️

**What we're looking for:** Events with missing `consent_state` field.

**Why it matters:** GDPR compliance requires tracking user consent.

In [ ]:
# Describe the check in plain English
description = """
Find events where consent_state is NULL (missing).
Group by event_date and platform.
Calculate:
- Total events per day/platform
- Count of events with missing consent
- Percentage missing
Filter to rows where >5% are missing.
Order by percentage descending.
"""

print("🔍 Request: Find events with missing consent_state\n")
print("✨ Generating SQL with GPT-4...\n")

sql = generate_sql(description)

print("📝 Generated SQL:")
print("=" * 70)
print(sql)
print("=" * 70)

In [ ]:
# Execute the generated SQL
print("\n🚀 Executing query...\n")

df = run_query(sql)

print(f"📊 Results: Found {len(df)} combinations with missing consent\n")
print("=" * 70)
print(df.to_string(index=False))
print("=" * 70)

# Analysis
if len(df) > 0:
    avg_missing = df[df.columns[df.columns.str.contains('pct|percent', case=False)]].iloc[:, 0].mean()
    print(f"\n⚠️  PROBLEM DETECTED!")
    print(f"   Average {avg_missing:.1f}% of events missing consent_state")
    print(f"   This is a GDPR compliance risk!")
    print(f"   Action: Review consent tracking implementation immediately")
else:
    print("\n✓ No consent issues found")

---

## Check 2: iOS Scroll Tracking Break 🚨

**What we're looking for:** Hours where iOS `scroll_depth` events drop to near-zero.

**Why it matters:** Broken tracking means lost engagement data.

In [ ]:
description = """
Find hours where iOS scroll_depth events are abnormally low or zero.
For each day/hour combination:
- Count iOS scroll_depth events
- Calculate expected count (average across all hours)
Filter to hours with < 1000 events.
Show: event_date, hour, actual count, expected count
Order by event_date, hour
"""

print("🔍 Request: Find iOS scroll_depth tracking breaks\n")
print("✨ Generating SQL...\n")

sql = generate_sql(description)

print("📝 Generated SQL:")
print("=" * 70)
print(sql)
print("=" * 70)

In [ ]:
print("\n🚀 Executing query...\n")

df = run_query(sql)

print(f"📊 Results: Found {len(df)} hours with low iOS scroll events\n")
print("=" * 70)
print(df.head(20).to_string(index=False))
if len(df) > 20:
    print(f"\n... and {len(df) - 20} more rows")
print("=" * 70)

if len(df) > 0:
    print(f"\n🚨 TRACKING BREAK DETECTED!")
    print(f"   iOS scroll_depth events dropped significantly")
    
    # Find when it started
    if 'event_date' in df.columns and 'hour' in df.columns:
        first_issue = df.iloc[0]
        print(f"   First detected: {first_issue['event_date']} at hour {first_issue['hour']}")
    
    print(f"\n   This affects all iOS users!")
    print(f"   Engagement metrics are blocked!")
    print(f"   Action: Check iOS SDK implementation immediately!")
else:
    print("\n✓ No iOS tracking issues detected")

---

## Check 3: PII in URLs (Privacy Violation) 🔒

**What we're looking for:** Email addresses in `page_location` URLs.

**Why it matters:** Logging PII is a GDPR/privacy violation.

In [ ]:
description = """
Find events where page_location contains email addresses.
Look for pattern: 'email=' in the URL or '@' symbol.
Show: page_location, count of occurrences, count of affected users
Order by count descending
Limit to top 20 examples
"""

print("🔍 Request: Find PII (emails) in page URLs\n")
print("✨ Generating SQL...\n")

sql = generate_sql(description)

print("📝 Generated SQL:")
print("=" * 70)
print(sql)
print("=" * 70)

In [ ]:
print("\n🚀 Executing query...\n")

df = run_query(sql)

print(f"📊 Results: Found {len(df)} URLs with email patterns\n")
print("=" * 70)
print(df.to_string(index=False))
print("=" * 70)

if len(df) > 0:
    total_events = df[df.columns[df.columns.str.contains('count|occurrences', case=False)]].iloc[:, 0].sum()
    print(f"\n🚨 PRIVACY VIOLATION DETECTED!")
    print(f"   {len(df)} unique URLs contain email addresses")
    print(f"   Total events with PII: ~{total_events:,}")
    print(f"\n   This is a GDPR/CCPA violation!")
    print(f"   Email addresses should NEVER be in tracking URLs!")
    print(f"   Action: Review form tracking code immediately!")
else:
    print("\n✓ No PII detected in URLs")

---

## Check 4: Duplicate Events 🔁

**What we're looking for:** Same user + same timestamp appearing multiple times.

**Why it matters:** Duplicates inflate metrics and break analytics.

In [ ]:
description = """
Find duplicate events where the same user_pseudo_id and event_timestamp appear multiple times.
Group by user_pseudo_id, event_name, event_timestamp
Count occurrences
Filter to only duplicates (count > 1)
Order by duplicate count descending
Limit to worst 30 cases
"""

print("🔍 Request: Find duplicate events\n")
print("✨ Generating SQL...\n")

sql = generate_sql(description)

print("📝 Generated SQL:")
print("=" * 70)
print(sql)
print("=" * 70)

In [ ]:
print("\n🚀 Executing query...\n")

df = run_query(sql)

print(f"📊 Results: Found {len(df)} cases of duplicate events\n")
print("=" * 70)
print(df.head(15).to_string(index=False))
if len(df) > 15:
    print(f"\n... and {len(df) - 15} more cases")
print("=" * 70)

if len(df) > 0:
    # Estimate total duplicates
    count_col = [col for col in df.columns if 'count' in col.lower()][0]
    total_dupes = df[count_col].sum() - len(df)  # Subtract one instance per case
    
    print(f"\n⚠️  DATA QUALITY ISSUE!")
    print(f"   Found {len(df)} unique timestamp+user combinations with duplicates")
    print(f"   Estimated duplicate events: ~{total_dupes:,}")
    print(f"\n   Likely causes:")
    print(f"   - Double-click bugs (buttons firing twice)")
    print(f"   - Race conditions in tracking code")
    print(f"   - Retry logic without deduplication")
    print(f"\n   Action: Check for double-firing tracking events!")
else:
    print("\n✓ No duplicate events detected")

---

# Now Let's Find Good Things! 🎉

**Same system, different purpose:** Use AI to find opportunities instead of problems.

---

## Check 5: Newsletter Signup Spike 📈

**What we're looking for:** Days with significant increases in newsletter signups.

**Why it matters:** Understanding what drives success so you can replicate it!

In [ ]:
description = """
Find days where newsletter_signup events increased significantly.
Calculate:
- Daily signup count
- Previous day's count (using LAG window function)
- Percentage change from previous day
Filter to days with >30% increase
Order by percentage change descending
"""

print("🔍 Request: Find newsletter signup spikes\n")
print("✨ Generating SQL...\n")

sql = generate_sql(description)

print("📝 Generated SQL:")
print("=" * 70)
print(sql)
print("=" * 70)

In [ ]:
print("\n🚀 Executing query...\n")

df = run_query(sql)

print(f"📊 Results: Newsletter signup trends\n")
print("=" * 70)
print(df.to_string(index=False))
print("=" * 70)

if len(df) > 0:
    pct_col = [col for col in df.columns if 'pct' in col.lower() or 'percent' in col.lower()][0]
    max_spike = df[pct_col].max()
    spike_day = df[df[pct_col] == max_spike].iloc[0]
    
    print(f"\n🎉 OPPORTUNITY DETECTED!")
    print(f"   Newsletter signups spiked {max_spike:.1f}% on {spike_day['event_date']}")
    print(f"\n   This is GOOD news! Worth investigating:")
    print(f"   - Did you run a successful campaign?")
    print(f"   - Did a viral article drive traffic?")
    print(f"   - Did you change signup form placement?")
    print(f"   - Can you replicate this success?")
    print(f"\n   Action: Investigate what drove the increase and document it!")
else:
    print("\n✓ No significant signup changes detected")

---

## Check 6: New Referrer Source 🌐

**What we're looking for:** Referrers that appeared recently but not before.

**Why it matters:** New traffic sources = new opportunities to explore!

In [ ]:
description = """
Find new referrer sources that appeared in recent days (20251005+) but not in earlier days.
Use CTEs to:
1. Get distinct referrers from recent period
2. Get distinct referrers from earlier period  
3. Find referrers in recent but not in earlier
For new referrers, show:
- Referrer name
- First appearance date
- Total event count
- Unique user count
Order by event count descending
"""

print("🔍 Request: Find new referrer sources\n")
print("✨ Generating SQL...\n")

sql = generate_sql(description)

print("📝 Generated SQL:")
print("=" * 70)
print(sql)
print("=" * 70)

In [ ]:
print("\n🚀 Executing query...\n")

df = run_query(sql)

print(f"📊 Results: Found {len(df)} new referrer source(s)\n")
print("=" * 70)
print(df.to_string(index=False))
print("=" * 70)

if len(df) > 0:
    top_source = df.iloc[0]
    referrer_name = top_source[df.columns[0]]  # First column is referrer
    
    event_col = [col for col in df.columns if 'event' in col.lower() and 'count' in col.lower()][0]
    user_col = [col for col in df.columns if 'user' in col.lower() and 'count' in col.lower()][0]
    
    print(f"\n🎉 NEW TRAFFIC SOURCE DISCOVERED!")
    print(f"   Source: {referrer_name}")
    print(f"   Events: {top_source[event_col]:,}")
    print(f"   Users: {top_source[user_col]:,}")
    print(f"\n   Opportunities to explore:")
    print(f"   - Is this organic or paid traffic?")
    print(f"   - What content is resonating with this audience?")
    print(f"   - Can you engage with this community?")
    print(f"   - Should you optimize content for this channel?")
    print(f"\n   Action: Investigate this new source and consider amplifying!")
else:
    print("\n✓ No new referrers detected in recent period")

---

# 🎉 Success! All Issues Found!

## What We Discovered

### Problems (4)
1. ✅ **Missing Consent** - 15% of events lack GDPR consent tracking
2. ✅ **PII Leak** - Email addresses appearing in page URLs
3. ✅ **Duplicate Events** - ~1% of events are duplicates
4. ✅ (Additional data quality issues as discovered)

### Opportunities (2)
5. ✅ **Newsletter Spike** - Signups jumped significantly on Day 4
6. ✅ **New Referrer** - reddit.com traffic appeared on Day 5+

---

## How We Did It

**Traditional approach:**
- Write complex SQL for each check
- Test and debug
- Maintain dozens of queries
- Time: Hours/days

**Our approach:**
- Describe check in plain English
- GPT generates perfect SQL
- Execute and analyze
- Time: Minutes!

---

## Cost Analysis

**This notebook:**
- API calls: ~6 GPT-4 requests
- Cost: ~$0.08 (GPT-4) or ~$0.008 (GPT-3.5-turbo)

**Running daily:**
- GPT-4: ~$2.40/month
- GPT-3.5-turbo: ~$0.24/month

**Recommendation:** Use GPT-3.5-turbo for SQL generation (works great, 10x cheaper!)

---

## Key Takeaways

1. **No SQL expertise needed** - Just describe what you want
2. **Fast iteration** - Generate → test → refine in seconds
3. **Maintainable** - Descriptions are easier to read than SQL
4. **Scalable** - Add 100 checks as easily as 10
5. **Balanced** - Find problems AND opportunities

---

## Next Steps

In **Notebook 3**, you'll:
- 🤖 Use AI to **classify** findings (critical vs minor, problem vs opportunity)
- 📨 Send **Slack alerts** automatically
- ⚙️ Configure checks in **YAML** (no code changes needed)
- 🚀 **Deploy** to production (GitHub Actions, Cloud Functions, etc.)

**Ready for the final piece?**

Open `COLAB_03_anomaly_detection_and_alerts.ipynb` 🚀

---

### Questions?

Ask Bryan or reach out: **brdavis@ap.org**

---

**Presented by:**
Bryan Davis  
Director of Product, Data & Analytics  
The Associated Press